<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/Cohere_and_Open_Source_Embedding_Model_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Packages and Setup Variables**

In [1]:
!pip install -q llama-index==0.10.57 openai==1.37.0 llama-index-finetuning llama-index-embeddings-huggingface llama-index-embeddings-cohere llama-index-readers-web cohere==5.6.2 tiktoken==0.7.0 chromadb==0.5.5 html2text sentence_transformers pydantic llama-index-vector-stores-chroma==0.1.10 llama-index-llms-gemini==0.1.11 llama-index-embeddings-instructor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.8 

In [13]:
import os
from google.colab import userdata

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] =  userdata.get('google_api_key')
os.environ["CO_API_KEY"] =  userdata.get('co_api_key')


In [14]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

We’ll use nest_asyncio to enable nested event loops in Python. This is particularly useful in scenarios where multiple asynchronous event loops must be run concurrently.

**Load a Model**

In [15]:
from llama_index.core import Settings
# For LLM

from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini") # this model is used as the default throughout the application.

**Load the Dataset (CSV)**

Download


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [16]:
!curl -o ./mini-llama-articles.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0  1597k      0 --:--:-- --:--:-- --:--:-- 1599k


**Read File**

In [17]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        rows.append(row)

# The number of characters in the dataset.
len(rows)

14

**Convert to Document obj**

In [18]:
from llama_index.core import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(
        text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}
    )
    for row in rows
]

# Cohere Embedding Model

### Creating Vector Store

In [19]:
import chromadb

# create a client and a new collection
# chromadb.EphemeralClient saves data in-memory.

chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

UniqueConstraintError: Collection mini-llama-articles already exists

In [20]:
from llama_index.vector_stores.chroma import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

**Transforming**

In [21]:
from llama_index.core.text_splitter import TokenTextSplitter

# Define the splitter object that split the text into segments with 512 tokens,
# with a 128 overlap between the segments.
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

There are two options to use the Cohere embeddings:

- input_type="search_document": Employ this option for texts (documents) intended for storage in your vector database.

- input_type="search_query": Use this when issuing search queries to locate the most related documents within your vector database.

In [23]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core.ingestion import IngestionPipeline


from llama_index.llms.openai import OpenAI
llm_gpt_4o = OpenAI(temperature=0, model="gpt-4o-mini")


# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm_gpt_4o),
        SummaryExtractor(summaries=["prev", "self"], llm=llm_gpt_4o),
        KeywordExtractor(keywords=10, llm=llm_gpt_4o),
        CohereEmbedding(model_name="embed-english-v3.0", input_type="search_document", cohere_api_key= userdata.get('co_api_key')),
    ],
    vector_store=vector_store,
)

# Run the transformation pipeline.
nodes = pipeline.run(documents=documents, show_progress=True)

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 108/108 [00:34<00:00,  3.14it/s]


Generating embeddings:   0%|          | 0/108 [00:00<?, ?it/s]

In [24]:
len(nodes)

108

In [25]:
len(nodes[0].embedding)

1024

In [26]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore_cohere.zip mini-llama-articles

  adding: mini-llama-articles/ (stored 0%)
  adding: mini-llama-articles/2f30ae9a-68f9-4e66-9bad-23d2d5701b9e/ (stored 0%)
  adding: mini-llama-articles/2f30ae9a-68f9-4e66-9bad-23d2d5701b9e/length.bin (deflated 54%)
  adding: mini-llama-articles/2f30ae9a-68f9-4e66-9bad-23d2d5701b9e/header.bin (deflated 61%)
  adding: mini-llama-articles/2f30ae9a-68f9-4e66-9bad-23d2d5701b9e/link_lists.bin (stored 0%)
  adding: mini-llama-articles/2f30ae9a-68f9-4e66-9bad-23d2d5701b9e/data_level0.bin (deflated 100%)
  adding: mini-llama-articles/chroma.sqlite3 (deflated 73%)


**Load Indexes**

If you have already uploaded the zip file for the vector store checkpoint, please uncomment the code in the following cell block to extract its contents. After doing so, you will be able to load the dataset from local storage.

In [27]:
# Load the vector store from the local storage.
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [28]:
from llama_index.core import VectorStoreIndex

# Define the Cohere Embedding Model
embed_model = CohereEmbedding( model_name="embed-english-v3.0", input_type="search_query", cohere_api_key= 'CO_API_KEY')

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store( vector_store, embed_model = embed_model)

**Query Dataset**

In [30]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine(similarity_top_k=5)

res = query_engine.query("How many parameters LLaMA2 model has?")

UnauthorizedError: status_code: 401, body: {'message': 'invalid api token'}

In [31]:
res.response

NameError: name 'res' is not defined

In [32]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

NameError: name 'res' is not defined

# Open Source Embedding

In [33]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Open Source Embedding Model Collection

chroma_client_ose = chromadb.PersistentClient(path="./mini-llama-articles-open-source-embed")
chroma_collection_ose = chroma_client_ose.create_collection("mini-llama-articles-open-source-embed")

# Define a storage context object using the created vector database.
vector_store_ose = ChromaVectorStore(chroma_collection=chroma_collection_ose)


In [ ]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import IngestionPipeline

from llama_index.llms.openai import OpenAI
llm_gpt_4o = OpenAI(temperature=0, model="gpt-4o-mini")

# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm_gpt_4o),
        SummaryExtractor(summaries=["prev", "self"], llm=llm_gpt_4o),
        KeywordExtractor(keywords=10, llm=llm_gpt_4o),
        HuggingFaceEmbedding(model_name="intfloat/e5-small-v2"),
    ],
    vector_store=vector_store_ose,
)

# Run the transformation pipeline.
nodes_ose = pipeline.run(documents=documents, show_progress=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

  6%|▌         | 6/108 [00:06<01:24,  1.21it/s]

In [ ]:
print(len(nodes_ose))
print(len(nodes_ose[0].embedding))

In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore_open_source_embed.zip mini-llama-articles-open-source-embed

In [ ]:
# Load the vector store from the local storage.
db_ose = chromadb.PersistentClient(path="./mini-llama-articles-open-source-embed")
chroma_collection_ose = db_ose.get_or_create_collection("mini-llama-articles-open-source-embed")
vector_store_ose = ChromaVectorStore(chroma_collection=chroma_collection_ose)

In [ ]:
from llama_index.core import VectorStoreIndex

# Define the Cohere Embedding Model
embed_model_e5 = HuggingFaceEmbedding(model_name="intfloat/e5-small-v2")

# Create the index based on the vector store.
index_ose = VectorStoreIndex.from_vector_store( vector_store_ose, embed_model = embed_model_e5)

# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine_ose = index_ose.as_query_engine(similarity_top_k=5)

res = query_engine_ose.query("How many parameters LLaMA2 model has?")

res.response

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)